In [ ]:
# default_exp pivoting

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

# pivoting

In order to create uniform datasets for the different primary financial statements (Income Statement, CaschFlow, BalanceSheet, ..) it is easier if the data is converted from its vertical for to a horizontal form. This means that we pivot the values based on the tag.

Therfore, for every primary financial statement a separate dataset is created.

Statement types

- IS: IncomeStatement
- CF: CashFlow
- BS: BalanceSheet
- CI: Comprehensive Income
- EQ: Equity
- CP: CoverPage
- UN: Unclassifiable Statement

## Basic Settings

In [ ]:
# imports
from bfh_mt_hs2020_sec_data.core import get_spark_session # initialze spark
from pathlib import Path
from typing import List, Tuple, Union, Set
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col

import pandas as pd

import shutil          # provides high level file operations
import time            # used to measure execution time
import os
import sys

In [ ]:
all_filtered_folder        = "D:/data/parq_filtered"      # source folder with the prepared parquet file
all_pivot_selected_folder  = "D:/data/parq_pivot_select"  # target folder which will contain only the needed subset of columns
all_pivoted_folder         = "D:/data/parq_pivot_split"   # target folder for the different pivoted and separated datasets

In [ ]:
# init Spark
spark = get_spark_session() # Session anlegen
spark # display the moste important information of the session

## 01_Load Data

**only execute if necessary**

Loads the and filtered data with all columns

In [ ]:
df_all = spark.read.parquet(all_filtered_folder).cache()

In [ ]:
# Expected Amount of Data
# Entries:  15_650_848
# duration:  66 sec

# load all data into memory
start = time.time()
print("Entries: ", "{:_}".format(df_all.count())) # loading all dataset into memory
duration = time.time() - start
print("duration: ", duration)

Entries:  15_650_848
duration:  90.72703671455383


## 02_Select

**only execute if necessary**

Creates a new dataset containing only  the columns that are needed during the next steps.

In [ ]:
from pyspark.sql.functions import year
df_all_selected = df_all.select(["stmt","cik","ticker", "adsh","period","form","filed","tag","value","report", "line", "fp", "uom", "qtrs",
                                year(df_all.period).alias('period_year')]).cache()

In [ ]:
df_all_selected = df_all_selected.where("period_year >= 2012")
print(df_all_selected.count())

13713772


In [ ]:
shutil.rmtree(all_pivot_selected_folder,  ignore_errors=True)
df_all_selected.write.parquet(all_pivot_selected_folder)

## 03_Pivoting

In [ ]:
df_all_selected = spark.read.parquet(all_pivot_selected_folder).cache()

In [ ]:
# Expected Amount of Data
# Entries:  15_650_848
# duration:  23 sec

# After removing qtrs 2 & 3
# Entries:  11_688_113
# duration:  11.602035999298096

# load all data into memory
start = time.time()
print("Entries: ", "{:_}".format(df_all_selected.count())) # loading all dataset into memory
duration = time.time() - start
print("duration: ", duration)

Entries:  15_650_848
duration:  28.721192836761475


In [ ]:
# if a value is null in the original data, we need to set it to zero, otherwise we cannot distinguish between a value that was or was not present 
# in the dataset after pivoting the data
df_all_selected = df_all_selected.fillna({'value':0.0}) # set null value in value column to 0.0

In [ ]:
pivot_attrs = ['value'] # column that contains the value which has to be pivoted

def pivot_statement(all_data_df, statement:str, stmtfilter:str):
    all_stmt_data = all_data_df.where("stmt == '" + statement + "'" + stmtfilter).cache()
    
    shutil.rmtree(all_pivoted_folder + "/" + statement,  ignore_errors=True)
    
    # it is important to also include the qrts,report and line into grupping. 
    # - for instance, the IS is often not just for the last quarter, but also for the numbers of quarters since beginning of the fiscal year
    # - the same tag can appear multiple times in a repor, e.g. the IS has cash at the beginning and at end of the period
    grouped_df = all_stmt_data.groupby(["cik","ticker","adsh","form","period","filed", "fp", "qtrs"])
    
    for attr in pivot_attrs: 
        # using max() is not the best approach. generally, a tag is only contained once in a report, but there are excptions
        # like the CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents in the CF statement.
        pivoted_df = grouped_df.pivot("tag").max(attr)

        # repartition(1), so that only one file is created
        pivoted_df.repartition(1).write.parquet(all_pivoted_folder + "/" + statement + "/" + attr) 
        
    all_stmt_data.unpersist()

In [ ]:
def pivot_statements(all_data_df, statements,filters):
    start = time.time()
    
    for stmt,stmtfilter in zip(statements,filters): 
        print (stmt, end = "")
        section_start = time.time()
        pivot_statement(all_data_df, statement=stmt,stmtfilter = stmtfilter)
        section_duration = time.time() - section_start
        print (": ", section_duration)

    duration = time.time() - start
    print("duration: ", duration)

## 99_Execution

In [ ]:
# df_is_reduced = df_all_selected.where("stmt == 'IS' and ((form == '10-K' and qtrs == '4') or (form == '10-Q' and qtrs == '1'))")
      
# pivot_statements(df_is_reduced, ["IS"])

In [ ]:
# section_start = time.time()
# pandas_df = df_is_reduced.toPandas()
# duration = time.time() - section_start
# print(duration)

In [ ]:
# df_all_selected.where("stmt == 'IS'").count()

In [ ]:
statements = ['IS','CF','CP','BS','CI','EQ','UN']

#in case of the IS statement, we are only interested in the qtrs=4 for a 10Ks and qtrs=1 for a 10Qs
filters = [
        " and ((form == '10-K' and qtrs == '4') or (form == '10-Q' and qtrs == '1'))",
        "",
        "",
        "",
        "",
        "",
        "",
]

# statements = ['IS']
# filters = [
#         " and ((form == '10-K' and qtrs == '4') or (form == '10-Q' and qtrs == '1'))"
# ]

pivot_statements(df_all_selected, statements, filters)

IS:  126.44597816467285
CF:  249.06622552871704
CP:  15.875993490219116
BS:  147.50100946426392
CI:  59.849002838134766
EQ:  67.31700253486633
UN:  57.290998458862305
duration:  723.5532269477844


In [ ]:
spark.stop()

In [ ]:
just_selected = df_all_selected.where("adsh = '0001558370-20-011113'").cache()
just_selected.show()

+----+-------+------+--------------------+----------+----+--------------------+----------+------+----+---+------+----+
|stmt|    cik|ticker|                adsh|    period|form|                 tag|     value|report|line| fp|   uom|qtrs|
+----+-------+------+--------------------+----------+----+--------------------+----------+------+----+---+------+----+
|  EQ|1528849|    RH|0001558370-20-011113|2020-07-31|10-Q|TreasuryStockShar...|     600.0|     6|  21| Q2|shares|   2|
|  EQ|1528849|    RH|0001558370-20-011113|2020-07-31|10-Q|TreasuryStockValu...|   72000.0|     6|  18| Q2|   USD|   2|
|  EQ|1528849|    RH|0001558370-20-011113|2020-07-31|10-Q|OtherComprehensiv...|  918000.0|     6|  23| Q2|   USD|   2|
|  EQ|1528849|    RH|0001558370-20-011113|2020-07-31|10-Q|OtherComprehensiv...| 3290000.0|     6|  23| Q2|   USD|   1|
|  EQ|1528849|    RH|0001558370-20-011113|2020-07-31|10-Q|StockIssuedDuring...| 7328000.0|     6|  16| Q2|   USD|   1|
|  EQ|1528849|    RH|0001558370-20-011113|2020-0

In [ ]:
just_is_sel = just_selected.where("stmt == 'IS'")

In [ ]:
grouped_df = just_is_sel.groupby(["cik","ticker","adsh","form","period","fp", "qtrs"])

In [ ]:
pivoted_df = grouped_df.pivot("tag").max('value')

In [ ]:
pd = pivoted_df.toPandas()

In [ ]:
pd

,cik,ticker,adsh,form,period,fp,qtrs,CostOfGoodsAndServicesSold,EarningsPerShareBasic,EarningsPerShareDiluted,...,IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,IncomeTaxExpenseBenefit,InterestExpense,NetIncomeLoss,OperatingIncomeLoss,OtherExpenses,RevenueFromContractWithCustomerIncludingAssessedTax,SellingGeneralAndAdministrativeExpense,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic
0,1528849,RH,0001558370-20-011113,10-Q,2020-07-31,Q2,17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1528849,RH,0001558370-20-011113,10-Q,2020-07-31,Q2,1,376863000.0,5.08,3.71,...,117302000.0,18879000.0,19418000.0,98423000.0,136568000.0,19266000.0,7.092820e+08,195851000.0,26564705.0,19386115.0
2,1528849,RH,0001558370-20-011113,10-Q,2020-07-31,Q2,2,660104000.0,4.93,3.75,...,112667000.0,17456000.0,39047000.0,95211000.0,172021000.0,59354000.0,1.192177e+09,360052000.0,25383730.0,19314479.0


In [ ]:
df_all_selected.shape

AttributeError: 'DataFrame' object has no attribute 'shape'